<a href="https://colab.research.google.com/github/PeeratPookpanich/text_summarization_hw/blob/main/TextSum_TFIDF_6410422017_Peerat_Pookpanich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DADS7203 Natural Language Processing
##6410422017 Peerat Pookpanich

## Import Data

In [ ]:
import base64
import requests

url = "https://raw.githubusercontent.com/PeeratPookpanich/text_summarization_hw/main/Transitioning%20from%20engineering%20to%20data%20science%20in%20Bangkok%20Thailand.txt"
article_content = requests.get(url)
article_content = article_content.text
print(article_content)

Data science is still the most attractive trend in Thailand nowadays. 
There are many open positions for data scientists, data engineers, and data-related roles compared to 2 years ago. 
I am one of the people who jumped into this field as well. I want to share with you guys my journey of switching into this field.

The decision

In 2017, I was a mechanical engineer who worked in the petrochemical factory in Rayong province, Thailand. 
Typically, I was making a presentation to the management team about why the machine failed or how to prevent the machine breakdown. 
At that time, I had a chance to utilize the data to ensure and answer most technical questions.

One day, I realized that Rayong is not a place for me to settle down. 
I wanted to go back to Bangkok, so I need to find a way to relocate myself. 
I was searching for a job, and it was hard to find a good one because my background is in mechanical engineering, and it is not that many good engineering jobs were placed in Bangkok

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import re
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Cleansing

In [ ]:
def _create_dictionary_table(text) -> dict:
  text = re.sub(r'(http|http(s)?://)?([\w-]+\.)+[\w-]+[.com|.in|.org]+(\[\?%&=]*)?','',text)
  text = re.sub(r'\<a href', ' ', text)
  text = re.sub(r'&amp;', '', text) 
  text = re.sub(r'\&\=\*\$\-', ' ', text)
  text = re.sub(r'[_";%(),!?:#@\[\]/]', '', text)
  text = re.sub(r'<br />', ' ', text)
  text = re.sub(r'\\', '', text)
  text = re.sub(r'/\n', '', text)
  # Removing stop words
  stop_words = set(stopwords.words("english"))
  words = word_tokenize(text)
  # Reducing words to their root form
  stem = PorterStemmer()
  # Creating dictionary for the word frequency table
  frequency_table = dict()
  for wd in words:
      wd = stem.stem(wd)
      if wd in stop_words:
          continue
      if wd in frequency_table:
          frequency_table[wd] += 1
      else:
          frequency_table[wd] = 1

  return frequency_table
#_create_dictionary_table(article_content)

**Tokenizing the article into sentences**


In [ ]:
nltk.download('punkt')
sentences = sent_tokenize(article_content)
sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Data science is still the most attractive trend in Thailand nowadays.',
 'There are many open positions for data scientists, data engineers, and data-related roles compared to 2 years ago.',
 'I am one of the people who jumped into this field as well.',
 'I want to share with you guys my journey of switching into this field.',
 'The decision\r\n\r\nIn 2017, I was a mechanical engineer who worked in the petrochemical factory in Rayong province, Thailand.',
 'Typically, I was making a presentation to the management team about why the machine failed or how to prevent the machine breakdown.',
 'At that time, I had a chance to utilize the data to ensure and answer most technical questions.',
 'One day, I realized that Rayong is not a place for me to settle down.',
 'I wanted to go back to Bangkok, so I need to find a way to relocate myself.',
 'I was searching for a job, and it was hard to find a good one because my background is in mechanical engineering, and it is not that many good eng

**Finding the weighted frequencies of the sentences**

In [ ]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    # Algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence in sentence_weight:
                    sentence_weight[sentence] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence] = frequency_table[word_weight]

        sentence_weight[sentence] = sentence_weight[sentence] / sentence_wordcount_without_stop_words
        #To ensure long sentences do not have unnecessarily high scores over short sentences
      
    return sentence_weight
_calculate_sentence_scores(sentences,_create_dictionary_table(article_content))

{'Data science is still the most attractive trend in Thailand nowadays.': 11.0,
 'There are many open positions for data scientists, data engineers, and data-related roles compared to 2 years ago.': 11.0,
 'I am one of the people who jumped into this field as well.': 15.714285714285714,
 'I want to share with you guys my journey of switching into this field.': 11.727272727272727,
 'The decision\n\nIn 2017, I was a mechanical engineer who worked in the petrochemical factory in Rayong province, Thailand.': 7.866666666666666,
 'Typically, I was making a presentation to the management team about why the machine failed or how to prevent the machine breakdown.': 7.266666666666667,
 'At that time, I had a chance to utilize the data to ensure and answer most technical questions.': 13.3,
 'One day, I realized that Rayong is not a place for me to settle down.': 9.3,
 'I wanted to go back to Bangkok, so I need to find a way to relocate myself.': 10.0,
 'I was searching for a job, and it was hard 

**Calculating the threshold of the sentences**
With this threshold, we can avoid selecting the sentences with a lower score than the average score.

In [ ]:
def _calculate_average_score(sentence_weight) -> int:
   
    # Calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    # Getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score

**Getting the summary**

In [ ]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''
    print(threshold)

    for sentence in sentences:
        if sentence in sentence_weight and sentence_weight[sentence] >= (threshold):
            article_summary += "\n" + sentence + str(sentence_weight[sentence])
            sentence_counter += 1

    return article_summary

In [ ]:
def _run_article_summary(article):
    
    #creating a dictionary for the word frequency table
    frequency_table = _create_dictionary_table(article)

    #tokenizing the sentences
    sentences = sent_tokenize(article)

    #algorithm for scoring a sentence by its words
    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)

    #getting the threshold
    threshold = _calculate_average_score(sentence_scores)

    #producing the summary
    article_summary = _get_article_summary(sentences, sentence_scores, 1.5 * threshold)
    #1.5 is hyperparameter 
    
    return article_summary

In [ ]:
summary_results = _run_article_summary(article_content)
print(summary_results)

15.867444678197028

As a result, I decide to switch from the engineering field to the data field.18.0
In Thailand, there were not many open data scientist positions.17.25
On the other hand, the downside is that data is never cleaned.17.285714285714285
Data engineering is the answer here to control the quality and integrity of the data pipeline.16.0


Reference: https://blog.floydhub.com/gentle-introduction-to-text-summarization-in-machine-learning/